In [1]:
import sqlite3
from Bio import SeqIO, SeqFeature, SeqRecord
from collections import defaultdict
import pickle
import glob
import os
import pandas
from tqdm.notebook import tqdm

In [ ]:
def pickle_dict(input_object, output_file):
    with open(output_file, 'wb') as jar:
        pickle.dump(input_object, jar)
    print(f"Successfully Pickled to {output_file}")

######################################################################################################
#::::::::::::::::::::::::::::::::::GENBANK PARSER AND DICT CREATOR:::::::::::::::::::::::::::::::::::#
######################################################################################################
# Lets set up our parsing function. This is just the barebones function.
# I'll expand it with additional helpers once I reach that point :)
# primarily to include the plasmid name for the replicon.and more?
def parse_genbank(file_path, assembly_id, contig2plasmid, id_parsing_dict=None):
    set_of_keys = set()
    ref_flag = True if assembly_id == "REF" else False
    if ref_flag and not id_parsing_dict:
        
    assemblies = SeqIO.parse(file_path, "genbank")
    genes = []
    for record in assemblies:
        if ref_flag == False and assembly_id not in contig2plasmid.keys():
            break
        for feature in record.features:
            if feature.type in ["gene", "CDS", "mRNA", "tRNA", "rRNA", "signal_peptide"]:
                if ref_flag:
                    assembly_id = id_parsing_dict[record.id]['strain']
                    replicon_name = id_parsing_dict[record.id]['name']
                else:
                    replicon_name = contig2plasmid_dict[assembly_id][record.id]['best_hit']

                locus_tag = feature.qualifiers.get("locus_tag", ['unknown'])[0]
                db_xrefs = feature.qualifiers.get("db_xref", [])
                db_xref_dict = {xref.split(":")[0]: xref.split(":")[1] for xref in db_xrefs}

                gene_info = {
                    "feature_type": feature.type,
                    "gene": feature.qualifiers.get("gene", ["unknown"])[0],
                    "locus_tag": locus_tag,
                    "note": feature.qualifiers.get("note", ["unknown"])[0],
                    "protein_id": feature.qualifiers.get("protein_id", ["unknown"])[0],
                    "product": feature.qualifiers.get("product", ["unknown"])[0],
                    "sequence": str(feature.extract(record.seq)),
                    "replicon": record.id,
                    "replicon_name": replicon_name,
                    "start": feature.location.start,
                    "end": feature.location.end,
                    "strand": feature.location.strand,
                    "assembly": assembly_id,
                    "translation": str(feature.qualifiers.get('translation',[''])),
                    "inference": feature.qualifiers.get("inference",[""])[0],
                    "transl_table": feature.qualifiers.get("transl_table",[""])[0],
                    'db_xrefs': feature.qualifiers.get("db_xrefs",[""])[0],
                    **db_xref_dict  # Unpack db_xref_dict to include db_xrefs as columns
                }
                genes.append(gene_info)
                set_of_keys.update(gene_info.keys())

    # fill out empty dicts with None!
    for gene in genes:
        for key in set_of_keys:
            if key not in gene:
                gene[key] = None

    return genes

In [4]:
######################################################################################################
#::::::::::::::::::::::::::::::HARD SET SCHEMA METHODS (First attempt):::::::::::::::::::::::::::::::#
######################################################################################################
def create_table(db_name, table_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      feature_type TEXT,
                      gene TEXT,
                      locus_tag TEXT,
                      note TEXT,
                      protein_id TEXT,
                      product TEXT,
                      sequence TEXT,
                      replicon TEXT,
                      replicon_name TEXT,
                      start INTEGER,
                      end INTEGER,
                      strand INTEGER,
                      db_xrefs TEXT,
                      assembly TEXT,
                      translation TEXT,
                      inference TEXT,
                      transl_table INT)''')
    conn.commit()
    conn.close()

def populate_table(db_name, table_name, genes):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    for gene in genes:
        cursor.execute(f'''INSERT INTO {table_name}
                          (feature_type, gene, locus_tag, note, protein_id, product, sequence, replicon, replicon_name, start, end, strand, db_xrefs, assembly, translation, inference, transl_table)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                       (gene["feature_type"], gene["gene"], gene["locus_tag"], gene["note"],
                        gene["protein_id"], gene["product"], gene["sequence"], gene["replicon"],
                        gene["replicon_name"], gene["start"], gene["end"], gene["strand"], gene["db_xrefs"],
                        gene["assembly"], gene["translation"], gene["inference"], gene["transl_table"]))
    conn.commit()
    conn.close()
######################################################################################################
#::::::::::::::::::::::::::::::AUTOMATED CREATION METHOD (2nd attempt):::::::::::::::::::::::::::::::#
######################################################################################################
# Make table and populate dynamically since we don't know all of the db_xrefs
# (or rather haven't used them in SCHEMA creation)
def create_and_populate_database(genes, db_name):
    df = pandas.DataFrame(genes)
    conn = sqlite3.connect(db_name)
    df.to_sql('annotations', conn, if_exists='replace', index=False)
    conn.close()


#####################################################################################################
#:::::::::::::::::::::::::::::::THESE HELPERS WORK WITH BOTH METHODS::::::::::::::::::::::::::::::::#
#####################################################################################################
def get_tables(db_name):
    conn = sqlite3.connect(db_name)
    tables = pandas.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    conn.close()
    return tables

def get_columns(db_name, table):
    # Describe the structure of the 'genes' table
    conn = sqlite3.connect(db_name)
    columns = pandas.read_sql_query(f"PRAGMA table_info({table});", conn)
    conn.close()
    return columns

def table_head(db_name, table):
    conn = sqlite3.connect(db_name)
    table_head = pandas.read_sql_query(f"SELECT * FROM {table} LIMIT 10;", conn)
    conn.close()
    return table_head

def get_all_of_gene(db_name, table, gene):
    conn = sqlite3.connect(db_name)
    all_genes = pandas.read_sql_query(f"SELECT * FROM {table} WHERE feature_type = 'CDS' AND gene='{gene}'", conn)
    conn.close()
    return all_genes

def get_all_of_product(db_name, table, product):
    conn = sqlite3.connect(db_name)
    all_products = pandas.read_sql_query(f"SELECT * FROM {table} WHERE feature_type = 'CDS' AND product='{product}'", conn)
    conn.close()
    return all_products

def dump_table_to_df(db_name, table):
    conn = sqlite3.connect(db_name)
    dataframe = pandas.read_sql_query(f"SELECT * FROM {table};", conn)
    conn.close()
    return dataframe

def connect_to_db(db_or_conn):
    # is our input a db path or a connection to a db?
    if isinstance(db_or_conn, str):
        # it is string, it must be a db path. Create a new connection
        conn = sqlite3.connect(db_or_conn)
        close_conn = True
    else:
        # Ain't a string so must be connection. Use the existing connection
        conn = db_or_conn
        close_conn = False
    return conn, close_conn

def get_gene_from_baktaID(db_or_conn, table, bakta_id):
    """
    example:
    print(get_gene_from_baktaID(db, 'assemblies', 'CKDGNL_04300'))
    """
    # check input db_or_conn and handle it.
    conn, close_conn = connect_to_db(db_or_conn)
    all_genes = pandas.read_sql_query(f"SELECT * FROM {table} WHERE locus_tag='{bakta_id}'", conn)
    # Pack it in pack it out!
    if close_conn:
        conn.close()
    return all_genes

# SQL should never be written more than once and here is my function.
def query_term_vs_multi_column(connection, table, term, columns):
    # Construct the SQL query
    query = f"""
    SELECT *
    FROM {table}
    WHERE {' OR '.join([f"{col} = ?" for col in columns])}
    """
    # try to find the term in each column.
    try:
        result_df = pandas.read_sql_query(query, connection, params=[term] * len(columns))
        #print(f"Query for term '{term}' succeeded.")
        return result_df # return our dataframe!
    except Exception as e:
        print(f"Query for term '{term}' failed with error: {e}")
        return pandas.DataFrame() # Empty on error.

In [56]:
######################################################################################################
#:::::::::::::::::::::::::::::::::::::::::::INPUT / OUTPUT:::::::::::::::::::::::::::::::::::::::::::#
######################################################################################################

# Let's hard set some paths, this will need to be changed and made flexible for remote execution :|
list_of_assemblies = glob.glob('../assemblies/dataset_v5/*/*.gbff') # Bakta genbanks for our assemblies!
bakta_reference_genbank = '/Users/mf019/bioinformatics/longread_pangenome/ref/all_plasmids_bbss/Bbss.gbff' # Bakta annotations of NCBI plasmid genomes. those in (wp_db)
list_of_ncbi_ref_genbanks = glob.glob('/Users/mf019/bioinformatics/longread_pangenome/plasmid_id/plasmid_seqs/raw_gbs/*.gb') # and the NCBI annotated genbanks
# Roary_v4 output
clustered_proteins_file = '/Users/mf019/bioinformatics/longread_pangenome/longread_analysis/v4/longread_paired_v4/clustered_proteins'
roary_gene_presence_absence_v4 = '/Users/mf019/bioinformatics/longread_pangenome/longread_analysis/v4/longread_paired_v4/gene_presence_absence.Rtab'
# Plasmid Calls
plasmid_calls_csv = '/Users/mf019/bioinformatics/longread_pangenome/plasmid_id/output/plasmid_calls_V7/best_matches_v7_1000bp.tsv'

# Sanity Check, (are my paths correct?)
# {{TODO: change these to assertions}}
print(len(list_of_assemblies))
print(len(list_of_ncbi_ref_genbanks))

# Define file paths for pickles
contig2plasmid_pickle = 'contig2plasmid_dict_1kb_v7.pkl'
assemblies_genbank_pickle = 'assembly_genbank_dict_v1.pkl'
ncbi_wp_genbank_pickle = 'ncbi_genbank_dict_v1.pkl'
bakta_wp_genbank_pickle = 'bakta_genbank_dict_v1.pkl'

# and now to define the jars with which to pickle our query results
gg2plasmid_presence_absence_pickle = 'gg2plasmid_presence_absence_dict_v1.pkl'
gg2plasmid_clustered_proteins_pickle = 'gg2plasmid_clustered_proteins_dict_v1.pkl'
groups_to_genelists_pickle = 'groups_to_IDs_clustered_proteins_dict_v1.pkl'


105
317


In [29]:
# let's parse our plasmid parsing dictionary using ole (somewhat)reliable.
# REMINDER: Structure is: {NCBI_ID : {'ID', 'name', 'length', 'strain'}}
#
with open('../../plasmid_id/blast/blast_parsing_dict.pickle', 'rb') as infile:
    id_parsing_dict = pickle.load(infile)

In [30]:
# Let's pull the plasmid calls in and  make us a dataframe
contig2plasmid_df = pandas.read_csv(plasmid_calls_csv, delimiter='\t')
# subset that to just what we care about (for now)
contig2plasmid_df = contig2plasmid_df[['name', 'contig', 'best_hit', 'completeness']]
# Bakta renamed contigs in each genbank, the calls were done from the original names in the original fasta.
# we just need to fix the scuffed contig names. This is simple.
contig2plasmid_df['new_contig'] = contig2plasmid_df.groupby('name').cumcount() + 1
contig2plasmid_df['new_contig'] = contig2plasmid_df['new_contig'].apply(lambda x: f'contig_{x}')
# and now let's flip it to a dict so we can use key lookups to simply rename them.
contig2plasmid_dict = defaultdict(lambda: defaultdict(dict))
for _, row in contig2plasmid_df.iterrows():
    contig2plasmid_dict[row['name']][row['new_contig']] = {
        'contig' : row ['contig'],
        'best_hit': row['best_hit'],
        'completeness': row['completeness']
    } # and we also key it to original contig name because bakta kept them for some assemblies (probably due to length)
    contig2plasmid_dict[row['name']][row['contig']] = {
        'contig' : row ['new_contig'],
        'best_hit': row['best_hit'],
        'completeness': row['completeness']
    }
contig2plasmid_dict = {name: dict(contigs) for name, contigs in contig2plasmid_dict.items()}

In [31]:
# and now let's pickle this so we can re-use it!
pickle_dict(contig2plasmid_dict, contig2plasmid_pickle)

Successfully Pickled: {'URI47': {'contig_1': {'contig': 'NODE_1_length_463767_cov_77.611941', 'best_hit': 'chromosome', 'completeness': 'incomplete'}, 'NODE_1_length_463767_cov_77.611941': {'contig': 'contig_1', 'best_hit': 'chromosome', 'completeness': 'incomplete'}, 'contig_2': {'contig': 'NODE_2_length_221463_cov_78.896272', 'best_hit': 'chromosome', 'completeness': 'incomplete'}, 'NODE_2_length_221463_cov_78.896272': {'contig': 'contig_2', 'best_hit': 'chromosome', 'completeness': 'incomplete'}, 'contig_3': {'contig': 'NODE_3_length_213568_cov_76.158203', 'best_hit': 'chromosome', 'completeness': 'incomplete'}, 'NODE_3_length_213568_cov_76.158203': {'contig': 'contig_3', 'best_hit': 'chromosome', 'completeness': 'incomplete'}, 'contig_4': {'contig': 'NODE_4_length_53880_cov_109.176347', 'best_hit': 'lp54', 'completeness': 'presumed'}, 'NODE_4_length_53880_cov_109.176347': {'contig': 'contig_4', 'best_hit': 'lp54', 'completeness': 'presumed'}, 'contig_5': {'contig': 'NODE_5_length_3

In [32]:
######################################################################################################
#::::::::::::::::::::::::::::::::::::::::::PARSE ASSEMBLIES::::::::::::::::::::::::::::::::::::::::::#
######################################################################################################
all_assembly_genes = []
total_feats = 0
for assembly in tqdm(list_of_assemblies, desc="Parsing assembly genbanks!"):
    current_genes = []
    assembly_id = os.path.basename(assembly).split('.')[0]
    current_genes = parse_genbank(assembly, assembly_id)
    num_feats = len(current_genes)
    total_feats += num_feats
    all_assembly_genes.append(current_genes)
all_assembly_genes_flat = [feature for features in all_assembly_genes for feature in features]
all_feats = len(all_assembly_genes_flat)
print("Making sure features match between input and final object")
print(f'list: {all_feats}')
print(f'actual: {total_feats}')
#pickle_dict(all_assembly_genes_flat, assemblies_genbank_pickle)

Parsing assembly genbanks!:   0%|          | 0/105 [00:00<?, ?it/s]

Making sure features match between input and final object
list: 268857
actual: 268857


In [33]:
######################################################################################################
#::::::::::::::::::::::::::::::::::::PARSE BAKTA ANNOTATIONS FOR REF:::::::::::::::::::::::::::::::::#
######################################################################################################
bakta_ref_genes = parse_genbank(bakta_reference_genbank, "REF")
print(len(bakta_ref_genes))
#pickle_dict(bakta_ref_genes,bakta_wp_genbank_pickle)

57558


In [34]:
######################################################################################################
#::::::::::::::::::::::::::::::::PARSE NCBI(gb) ANNOTATIONS FOR REF::::::::::::::::::::::::::::::::::#
######################################################################################################
all_ref_ncbi_genes = []
total_feats = 0
for plasmid in list_of_ncbi_ref_genbanks:
    current_genes = []
    current_genes = parse_genbank(plasmid, "REF")
    num_feats = len(current_genes)
    total_feats += num_feats
    all_ref_ncbi_genes.append(current_genes)
all_ref_ncbi_genes_flat = [feature for features in all_ref_ncbi_genes for feature in features]
all_feats = len(all_ref_ncbi_genes_flat)
print(f'list: {all_feats}')
print(f'actual: {total_feats}')
#pickle_dict(all_ref_ncbi_genes_flat, ncbi_wp_genbank_pickle)

list: 58093
actual: 58093


In [36]:
db = "Bbss_db_v1.db"
new_db = "Bbss_db_v2.db"
create_table(db, "assemblies")
populate_table(db,"assemblies", all_assembly_genes_flat)
create_table(db, "reference")
populate_table(db,"reference", bakta_ref_genes)
create_table(db, "ncbi_gb")
populate_table(db, "ncbi_gb", all_ref_ncbi_genes_flat)

# List all tables
#print(get_tables(db))
## Describe the structure of the 'genes' table
#print(get_columns(db, "assemblies"))
## Query some data from the 'genes' table
#print(table_head(db, "assemblies"))
## Describe the structure of the 'reference' table
#print(get_columns(db, "reference"))
## Query some data from the 'reference' table
#print(table_head(db, "reference"))
## Describe the structure of the 'reference' table
#print(get_columns(db, "ncbi_gb"))
## Query some data from the 'reference' table
#print(table_head(db, "ncbi_gb"))

# Let us make a new database just off the assemblies with db_xref parsed out.
create_and_populate_database(all_assembly_genes_flat, new_db)

In [60]:
get_tables(new_db)

,name
0,annotations


In [37]:
specific_genes = ['ospC', 'cspZ', 'cspA']
#all_cspA_df = get_all_of_gene(db, "assemblies", "cspA") # IT IS NOT ANNOTATED AS CSPA SO THANKS NCBI!
all_cspA_df = get_all_of_product(db, "assemblies", "complement regulator-acquiring protein")
all_cspA_df.to_csv('all_cspA_df.csv', index=False)
all_cspZ_df = get_all_of_gene(db, "assemblies", "cspZ")
all_cspZ_df.to_csv('all_cspZ_df.csv', index=False)
all_ospC_df = get_all_of_gene(db, "assemblies", "ospC")
all_ospC_df.to_csv('all_ospC_df.csv', index=False)

In [38]:
pangenome_df  = pandas.read_csv(roary_gene_presence_absence_v4, delimiter='\t')
pangene_groups = pangenome_df['Gene'].to_list()

In [39]:
columns_to_check = ['product', 'locus_tag', 'gene']
group_dfs = []
searched_groups = []
list_of_unresolved_groups = []

# Establish a connection to the database
connection = sqlite3.connect(new_db)
cursor = connection.cursor()

# ok lets start this loop now.
for group in tqdm(pangene_groups, desc="Searching for those groups!"):
    if "group" in group: # pull out the obvious groups that won't be found.
        list_of_unresolved_groups.append(group) # put em in a list that we're gonna have to approach differently
    else:
        group_df = query_term_vs_multi_column(connection, "annotations", group, columns_to_check)
        group_dfs.append(group_df)
        searched_groups.append(group)

# Close the connection
connection.close()
print(len(list_of_unresolved_groups))

Searching for those groups!:   0%|          | 0/3921 [00:00<?, ?it/s]

2932


In [59]:
groups_to_genelists = defaultdict(list)
with open(clustered_proteins_file, 'r') as infile:
    for line in infile.readlines():
        splitline = line.split('\t')
        group_id = splitline[0].split(':')[0]
        gene1 = [splitline[0].split(':')[1].strip(" ").strip('\n')]
        rest_of_genes = [gene.strip('\n') for gene in splitline[1::]] if len(splitline) > 1 else []
        groups_to_genelists[group_id] = gene1
        groups_to_genelists[group_id].extend(rest_of_genes)
print(len(groups_to_genelists.keys()))

pickle_dict(groups_to_genelists, groups_to_genelists_pickle)

3921
Successfully Pickled to groups_to_IDs_clustered_proteins_dict_v1.pkl


In [ ]:
# Establish a connection to the database
connection = sqlite3.connect(new_db)
cursor = connection.cursor()

all_groups_df = defaultdict(dict)
# ok lets start this loop now.
for group in tqdm(groups_to_genelists.keys(), desc=f"querying groups!"):
    all_groups_df[group] = []
    genes = groups_to_genelists[group]
    group_dfs = pandas.DataFrame()
    for gene in genes:
        group_df = get_gene_from_baktaID(connection, "annotations", gene)
        group_dfs = pandas.concat([group_dfs, group_df])

        # Concatenate the DataFrames for this group
    if not group_dfs.empty:
        all_groups_df[group].append(group_dfs)

# Close the connection
connection.close()